In [311]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime

In [312]:
page_address_list=["https://www.immoregion.fr/vente/maison/metz/id-6708871.html","https://www.immoregion.fr/vente/maison/metz/id-6708870.html","https://www.immoregion.fr/vente/maison/saint-julien-les-metz/id-6696596.html","https://www.immoregion.fr/vente/maison/metz/id-6616097.html"]

In [321]:
json_list=[]
for link in seed_address_list:
    page=requests.get(link)
    if page.status_code==200:
        page_content=BeautifulSoup(page.content,"html.parser")

        price=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[0].find_all("div")[0].text
        area=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[3].find_all("div")[0].text
        furnished=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[2].find_all("div")[0].text
        integral_area=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[4].find_all("div")[0].text
        nmbr_of_rooms=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[5].find_all("div")[0].text
        ref=page_content.find_all("h1",class_="KeyInfoBlockStyle__PdpTitle-sc-1o1h56e-2 hWEtva")[0].find_all("span")[0].text
        description=page_content.find_all("div",class_="DescriptionStyle__DescriptionContentDropdown-dqxmgt-3 dyQEIL")[0].find_all("div")[0].text
        link_address=link
        scrapping_time=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        agency=page_content.find_all("h3",class_="FormAgencyInfoStyle__AgencyName-sc-10hhmvq-5 jfZIgK")[0].text
        address=page_content.find_all("div",class_="block-localisation-address")[0].text

        json_data={"Ref_no":ref,"Price":price,"Surface":area,"furnished":furnished,"integral_area":whole_area,"No_room":nmbr_of_rooms,"Description":description,"Link":link_address,"scrapping_time":scrapping_time,"agency":agency,"address":address}
        json_list.append(json_data)
    else:
        pass

In [322]:
import pandas as pd
pd.DataFrame(json_list)

,Ref_no,Price,Surface,furnished,integral_area,No_room,Description,Link,scrapping_time,agency,address
0,Réf 6708871,310 000 €,"88,65 m²",Non,110 m²,2,Ce bien se trouve dans ce LotissementMetz Val...,https://www.immoregion.fr/vente/maison/metz/id...,2020-05-26 17:58:53,Laforet - Metz,rue Jeanne Jugan - Metz
1,Réf 5588552,310 000 €,"88,65 m²",NC,110 m²,4,Ce bien se trouve dans ce LotissementProgramm...,https://www.immoregion.fr/vente/maison/metz/id...,2020-05-26 17:58:54,Laforet - Metz,57000 MetzL'adresse exacte de ce bien n'est p...
2,Réf 6697900,307 000 €,NC,2020-03-30,110 m²,96 m²,Ce bien se trouve dans ce Lotissement,https://www.immoregion.fr/vente/maison/saint-j...,2020-05-26 17:58:54,DELAVILLE IMMOBILIER,57070 Saint-Julien-lès-MetzL'adresse exacte d...
3,Réf 6696596,307 000 €,Détails,NC,110 m²,5,"SAINT JULIEN LES METZ: Sur les hauteurs, pavil...",https://www.immoregion.fr/vente/maison/saint-j...,2020-05-26 17:58:54,DELAVILLE IMMOBILIER,57070 Saint-Julien-lès-MetzL'adresse exacte d...
4,Réf 6708871,310 000 €,"88,65 m²",Non,110 m²,2,Ce bien se trouve dans ce LotissementMetz Val...,https://www.immoregion.fr/vente/maison/metz/id...,2020-05-26 17:58:55,Laforet - Metz,rue Jeanne Jugan - Metz
5,Réf 6697900,307 000 €,NC,2020-03-30,110 m²,96 m²,Ce bien se trouve dans ce Lotissement,https://www.immoregion.fr/vente/maison/saint-j...,2020-05-26 17:58:55,DELAVILLE IMMOBILIER,57070 Saint-Julien-lès-MetzL'adresse exacte d...
6,Réf 6696596,307 000 €,Détails,NC,110 m²,5,"SAINT JULIEN LES METZ: Sur les hauteurs, pavil...",https://www.immoregion.fr/vente/maison/saint-j...,2020-05-26 17:58:55,DELAVILLE IMMOBILIER,57070 Saint-Julien-lès-MetzL'adresse exacte d...
7,Réf 6712160,309 000 €,NC,de l'acquéreur,110 m²,115 m²,Maison Saint-Julien-Les-Metz 5 pièces 115 m2. ...,https://www.immoregion.fr/vente/maison/saint-j...,2020-05-26 17:58:56,Benedic SAS,57070 Saint-Julien-lès-MetzL'adresse exacte d...
8,Réf 6616097,299 900 €,110 m²,NC,110 m²,6,Maison Metz Queuleu :\nCharmante maison mitoye...,https://www.immoregion.fr/vente/maison/metz/id...,2020-05-26 17:58:56,Centrum Immobilier,57000 MetzL'adresse exacte de ce bien n'est p...
9,Réf 6696596,307 000 €,Détails,NC,110 m²,5,"SAINT JULIEN LES METZ: Sur les hauteurs, pavil...",https://www.immoregion.fr/vente/maison/saint-j...,2020-05-26 17:58:56,DELAVILLE IMMOBILIER,57070 Saint-Julien-lès-MetzL'adresse exacte d...


In [ ]:
"/vente/maison/metz/id-5824233.html"

In [323]:
#get new links which are similar to the one you have looked at

seed_address="https://www.immoregion.fr/vente/maison/metz/id-6708871.html"
seed_address_list=[]

for i in range(0,3,1):
    if len(seed_address_list)==0:
        #add original seed_address
        print("seed_address_list is empty")
        seed_address_list.append(seed_address) 
        #read the original seed address and if reading process successful load html in var pagecontent
        page=requests.get(seed_address)                 
        if page.status_code==200:
            print("seed page loaded")
            page_content=BeautifulSoup(page.content,"html.parser")
            #iterate over 10 found links to other offers
            for link in page_content.find_all("a",{"href" : re.compile('/vente/maison/*/')})[-3:]:
                new_link="https://www.immoregion.fr"+link["href"]
                seed_address_list.append(new_link)
                print("link from seed_page added")
                print(new_link)
            counter=1
        else:
            counter=1
        print("erste Schleife zuende")
        print(seed_address_list)
        print("\n")    
    else:
        print("seed_address not empty")
        print("counter: {}".format(counter))
        for link in seed_address_list[counter:]:
            page=requests.get(link)
            if page.status_code==200:
                page_content=BeautifulSoup(page.content,"html.parser")
                #iterate over n found links to other offers
                print("Länge Link Liste: {}".format(len(page_content.find_all("a",{"href" : re.compile('/vente/maison/*/')}))))
                for link in page_content.find_all("a",{"href" : re.compile('/vente/maison/*/')})[-3:]:
                    new_link="https://www.immoregion.fr"+link["href"]
                    new_links.append(new_link)
                    print("link from seed_page added")
                    print(new_link)

            else:
                pass
        print("\n")
        [seed_address_list.append(address) for address in new_links]
        print("seed_address_list updated")
        print("Länge Seed address_list: {}".format(len(seed_address_list)))
        counter+=3
        print("Counter updated: {}".format(counter))
        new_links=[]
        print("\n")
        
        

seed_address_list is empty
seed page loaded
link from seed_page added
https://www.immoregion.fr/vente/maison/metz/id-5588552.html
link from seed_page added
https://www.immoregion.fr/vente/maison/saint-julien-les-metz/id-6696596.html
link from seed_page added
https://www.immoregion.fr/vente/maison/saint-julien-les-metz/id-6697900.html
erste Schleife zuende
['https://www.immoregion.fr/vente/maison/metz/id-6708871.html', 'https://www.immoregion.fr/vente/maison/metz/id-5588552.html', 'https://www.immoregion.fr/vente/maison/saint-julien-les-metz/id-6696596.html', 'https://www.immoregion.fr/vente/maison/saint-julien-les-metz/id-6697900.html']


seed_address not empty
counter: 1
Länge Link Liste: 12
link from seed_page added
https://www.immoregion.fr/vente/maison/metz/id-6708871.html
link from seed_page added
https://www.immoregion.fr/vente/maison/saint-julien-les-metz/id-6696596.html
link from seed_page added
https://www.immoregion.fr/vente/maison/saint-julien-les-metz/id-6697900.html
Länge 